# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) Carregar arquivo e verificar se existe registro duplicado
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)

# Caso exista, exlui os duplicados.
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) Conte o número de valores missing por variável.
sinasc.isna().sum()


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, 
# muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas 
# que interessam. São elas: ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 
# 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] Refaça a contagem de valores missings.

sinasc_red = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_red.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [4]:
sinasc_red.dtypes

LOCNASC         int64
IDADEMAE        int64
ESTCIVMAE     float64
ESCMAE         object
QTDFILVIVO    float64
GESTACAO       object
GRAVIDEZ       object
CONSULTAS       int64
APGAR5        float64
dtype: object

In [5]:
# 4) Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características 
# associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. 
# Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 
# não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.

# remove valores nulos da coluna APGAR5
sinasc_red = sinasc_red.dropna(subset=['APGAR5'])
sinasc_red.isna().sum()


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [6]:
# 5) observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. 
# Vamos assumir que o não preenchido é o mesmo que o código 9.

sinasc_red['ESTCIVMAE'].value_counts()

2.0    9906
1.0    8907
5.0    7145
4.0     451
9.0     144
3.0      57
Name: ESTCIVMAE, dtype: int64

In [7]:
sinasc_red.loc[sinasc_red['ESTCIVMAE'].isna(),['ESTCIVMAE']] = 9.0

In [8]:
sinasc_red['ESTCIVMAE'].isna().sum()

0

In [9]:
sinasc_red[['ESTCIVMAE']].value_counts()

ESTCIVMAE
2.0          9906
1.0          8907
5.0          7145
9.0           459
4.0           451
3.0            57
dtype: int64

A coluna 'CONSULTAS' não possui valores NA

In [10]:
sinasc_red[['CONSULTAS']].isna().sum()

CONSULTAS    0
dtype: int64

In [11]:
sinasc_red['CONSULTAS'].value_counts()

4    18587
3     5957
2     1359
1      972
9       50
Name: CONSULTAS, dtype: int64

In [12]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
sinasc_red['QTDFILVIVO'].value_counts()

1.0     9361
0.0     9305
2.0     4454
3.0     1362
4.0      475
5.0      212
6.0      106
7.0       41
8.0       23
9.0        8
10.0       6
12.0       3
11.0       1
30.0       1
14.0       1
Name: QTDFILVIVO, dtype: int64

In [13]:
sinasc_red[['QTDFILVIVO']].isna().sum()

QTDFILVIVO    1566
dtype: int64

In [14]:
sinasc_red.loc[sinasc_red['QTDFILVIVO'].isna(),['QTDFILVIVO']] = 0
sinasc_red['QTDFILVIVO'].value_counts()

0.0     10871
1.0      9361
2.0      4454
3.0      1362
4.0       475
5.0       212
6.0       106
7.0        41
8.0        23
9.0         8
10.0        6
12.0        3
11.0        1
30.0        1
14.0        1
Name: QTDFILVIVO, dtype: int64

In [15]:
sinasc_red[['QTDFILVIVO']].isna().sum()

QTDFILVIVO    0
dtype: int64

In [16]:
# 7) Das restantes, decida que valores te parece mais adequado (um 'não preenchido' ou um valor 
# 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados 
# é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente 
# certo e errado aqui.

sinasc_red.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [17]:
sinasc_red['ESCMAE'].value_counts()

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: ESCMAE, dtype: int64

No dicionário de dados temos uma opção 'Ignorado' para o campo Escolaridade da Mãe, vamos utilizar esta opção para os valores NA.

In [18]:
sinasc_red.loc[sinasc_red['ESCMAE'].isna(), ['ESCMAE']] = 'Ignorado'
sinasc_red[['ESCMAE']].isna().sum()

ESCMAE    0
dtype: int64

In [19]:
sinasc_red['GESTACAO'].value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

No dicionário de dados temos uma opção 'Ignorado' para o campo GESTACAO, vamos utilizar esta opção para os valores NA.

In [20]:
sinasc_red.loc[sinasc_red['GESTACAO'].isna(), ['GESTACAO']] = 'Ignorado'
sinasc_red[['GESTACAO']].isna().sum()

GESTACAO    0
dtype: int64

No dicionário de dados temos uma opção 'Ignorado' para o campo GRAVIDEZ, vamos utilizar esta opção para os valores NA.

In [21]:
sinasc_red.loc[sinasc_red['GRAVIDEZ'].isna(), ['GRAVIDEZ']] = 'Ignorado'
sinasc_red[['GRAVIDEZ']].isna().sum()

GRAVIDEZ    0
dtype: int64

In [22]:
sinasc_red.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [23]:
# 8) O Apgar possui uma classificação indicando se o bebê passou por asfixia:
# Entre 8 e 10 está em uma faixa 'normal'.
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
# Entre 4 e 5 significa 'asfixia moderada'.
# Entre 0 e 3 significa 'asfixia severa'.
# Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

sinasc_red.loc[sinasc_red['APGAR5'] >= 8 , 'apgar_categ'] = 'normal'
sinasc_red.loc[(sinasc_red['APGAR5'] >= 6) & (sinasc_red['APGAR5'] < 8), 'apgar_categ'] = 'asfixia leve'
sinasc_red.loc[(sinasc_red['APGAR5'] >= 4) & (sinasc_red['APGAR5'] < 6), 'apgar_categ'] = 'asfixia moderada'
sinasc_red.loc[sinasc_red['APGAR5'] < 4, 'apgar_categ'] = 'asfixia severa'


In [24]:
sinasc_red['apgar_categ'].value_counts()

normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: apgar_categ, dtype: int64

In [25]:
# 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um 
# underscore entre as palávras. Dica: repare que se você não quiser criar um dataframe novo, você vai
# precisar usar a opção inplace = True.

sinasc_red.columns = [['local_nascimento', 'idade_mae', 'estado_civil_mae', 'escolaridade_mae',
                      'qtd_filho_vivo', 'tempo_gestacao', 'tipo_gravidez', 'numero_consultas',
                      'apgar', 'apgar_categoria']]
sinasc_red.head()


,local_nascimento,idade_mae,estado_civil_mae,escolaridade_mae,qtd_filho_vivo,tempo_gestacao,tipo_gravidez,numero_consultas,apgar,apgar_categoria
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
